In [ ]:
import glob
import random
import os
import pandas as pd
import matplotlib.image as mpimg

In [ ]:
import platform
platform.system()

if platform.system() == 'Darwin':
    is_mac = True

In [ ]:
if not is_mac:
    !pip install pycaret

In [ ]:
if not is_mac:
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
if is_mac:
    image_path = '/Users/Mark/Google Drive/My Drive/Reef Support Benthic Data/SEAVIEW/PAC_AUS/'
    annot_path = '/Users/Mark/Google Drive/My Drive/Reef Support Benthic Data/SEAVIEW/tabular-data/'
else:
    image_path = '/content/gdrive/MyDrive/Reef Support Benthic Data/SEAVIEW/PAC_AUS/'
    annot_path = '/content/gdrive/MyDrive/Reef Support Benthic Data/SEAVIEW/tabular-data/'

In [ ]:
annot_file = 'annotations_PAC_AUS.csv'
df = pd.read_csv(annot_path + annot_file, index_col=0)
df.shape

In [ ]:
# sometimes an x,y label coordinate is outside the allowable values (based on the image dimensions)
df.drop(index=[13002079901, 13002086701, 15033259202], inplace=True)
df.shape

In [ ]:
df.index.unique().shape

In [ ]:
# select N images randomly 
N = 500

keep = random.sample(df.index.unique().to_list(), N)
df = df.loc[keep]
df.shape

In [ ]:
# create list of images to read from ids in label file
ids = df.index.unique().to_list()
image_files = [image_path + str(id) + '.jpg' for id in ids]
len(ids)

In [ ]:
df[['r','g','b']] = int(0)

for ctr, image_file in enumerate(image_files):
    id = int(os.path.basename(image_file).rstrip('.jpg'))
    #print(image_file, id)
    img = mpimg.imread(image_file)
    df.loc[id,['r','g','b']] = img[df.loc[id]['y'], df.loc[id]['x'], :] 
    #df.loc[id,['r','g','b']] = img[df.loc[id]['x'], df.loc[id]['y'], :] 
    print(f'{ctr+1}/{len(image_files)}',end='\r')

In [ ]:
df.head()

In [ ]:
df['is_coral'] = df['func_group'].str.contains('Coral')

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.drop(columns=['quadratid','y','x','label_name','label','func_group','method','data_set'], inplace=True)

In [ ]:
data = df.sample(frac=0.95, random_state=786)
data_unseen = df.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
from pycaret.classification import *

In [ ]:
pyc = setup(data=data, target='is_coral', session_id=123, silent=True)

In [ ]:
best_model = compare_models()

In [ ]:
print(best_model)

In [ ]:
rf = create_model('rf')

In [ ]:
print(rf)

In [ ]:
tuned_rf = tune_model(rf)

In [ ]:
print(tuned_rf)

In [ ]:
plot_model(tuned_rf, plot = 'confusion_matrix')

In [ ]:
plot_model(tuned_rf, plot='feature')

In [ ]:
predict_model(tuned_rf);